In [1]:
!git clone https://github.com/NVIDIA/cuda-samples.git

Cloning into 'cuda-samples'...
remote: Enumerating objects: 25798, done.
remote: Counting objects: 100% (6376/6376), done.
remote: Compressing objects: 100% (1109/1109), done.
remote: Total 25798 (delta 5754), reused 5278 (delta 5267), pack-reused 19422 (from 3)
Receiving objects: 100% (25798/25798), 134.42 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (22437/22437), done.
Updating files: 100% (2498/2498), done.


## CUDA Utilities


In [2]:
%%writefile cuda_stuff.cuh
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#ifndef cuda_stuff_H
#define cuda_stuff_H

/* transform matrix index to vector offset
   Since CUDA uses column major,
   nb_rows = number of rows */
#define IDX2C(i,j,nb_rows) (((j)*(nb_rows))+(i))

//MACRO TO DEBUGG CUDA FUNCTIONS
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

/** Error checking for use with CUDA Dynamic Parallelism */
/*
#define cdpErrchk(ans) { cdpAssert((ans), __FILE__, __LINE__); }
__device__ void cdpAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      printf("GPU kernel assert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) assert(0);
   }
}
*/

void device_synchronize();

#endif


Writing cuda_stuff.cuh


In [3]:
%%writefile cuda_stuff.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include "cuda_stuff.cuh"

void device_synchronize(){
    gpuErrchk(cudaDeviceSynchronize());
}

Writing cuda_stuff.cu


# The fmatrix Data Structure


In [4]:
%%writefile fmatrix.cuh
#ifndef fmatrices_H
#define fmatrices_H
#include "cuda_stuff.cuh" // for IDX2C

////////////////////////////////////////
// basic data structure and access macro
////////////////////////////////////////
typedef struct {
    float* data;
    int cols;
    int rows;
} fmatrix;

/** Access element (i,j) of matrix M
 *
 *  Usage example:
 *  For computing A = B^T + C, loop over i and j with:
 *    getfm(A,i,j) = getfm(B,j,i) + getfm(C,i,j);
 **/
#define getfm(M,i,j) (M.data[IDX2C(i,j,M.rows)])

////////////////////////////////////////
// utility functions
////////////////////////////////////////
/** Returns the number of elements in the matrix.
 *
 *  Useful for computing, e.g., the size
 *  of a 1D-vector that contains the same numbers.
 */
 __host__
 __device__
int fmatrix_elements(fmatrix mat);

/** Returns the memory occupied by the matrix elements in bytes
 *  (not including the variables in the struct mat).
 *
 *  Useful for allocating memory for the data.
 */
 __host__
 __device__
int fmatrix_size(fmatrix mat);

/** Assert that the matrix is coherent: all fields nonzero. */
 __host__
 __device__
void fmatrix_assert(fmatrix mat);

////////////////////////////////////////
// Create, copy, destroy
////////////////////////////////////////
/** Allocate memory on host */
fmatrix fmatrix_create_on_host(int rows, int cols);

/** Allocate memory on device */
fmatrix fmatrix_create_on_device(int rows, int cols);

/** Create a matrix representing columns [a,b) of M.
 *  Note that the new matrix uses a pointer to the
 *  data of M. The data is not copied to a new location.
 *  If M is destroyed, this matrix is useless.
 */
fmatrix fmatrix_subcolumns(fmatrix M, int a, int b);

/** Copy data from matrix on device to host
 *  (no memory allocation). */
void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device);

/** Copy data from matrix on host to device
 *  (no memory allocation). */
void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device);

/** Copy matrix from device to host, allocating new memory. */
fmatrix fmatrix_copy_to_host(fmatrix mat_device);

/** Copy matrix from host to device, allocating new memory. */
fmatrix fmatrix_copy_to_device(fmatrix mat_host);

/** Free data memory on host.
 *  This zeros out the data pointer of the fmatrix struct,
 *  so a pointer is required. */
void fmatrix_free_on_host(fmatrix* mat);

/** Free data memory on device.
 *  This zeros out the data pointer of the fmatrix struct,
 *  so a pointer is required. */
void fmatrix_free_on_device(fmatrix* mat);

////////////////////////////////////////
// Input and Output
////////////////////////////////////////

/** Print the first nb rows of the matrix mat
 *  on the host.
 *  If nb<0, print all rows.
 */
 __host__
 __device__
void fmatrix_print(fmatrix mat, int nb=-1);

/** Print the first nb rows of the matrix mat
 *  on the device.
 *  If nb<0, print all rows.
 *
 *  This version copies the matrix to host first.
 */
void fmatrix_device_print(fmatrix mat, int nb=-1);

/** Print a matrix to a csv file.
 *
 *  This version copies the matrix to host first.
 */
void fmatrix_device_to_csv(const char* filename, fmatrix mat);

/** Read a matrix from a csv file.
 *
 *  This version creates the matrix on the host first.
 */
fmatrix fmatrix_device_from_csv(const char* filename);

////////////////////////////////////////
// Useful
////////////////////////////////////////

/** Create a matrix with random values between -1 and 1
 *  on the device */
fmatrix fmatrix_create_random_on_device(int rows, int cols);

#endif


Writing fmatrix.cuh


In [5]:
%%writefile fmatrix.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <curand.h>
#include <curand_kernel.h>
#include "cuda_stuff.cuh"
#include "fmatrix.cuh"

// for reading CSV files, we use some C++
#include <iostream>
#include <iomanip>
#include <fstream>
#include <string>

int fmatrix_elements(fmatrix mat) {
     return mat.cols*mat.rows;
}

int fmatrix_size(fmatrix mat) {
     return fmatrix_elements(mat) * sizeof(float);
}

void fmatrix_assert(fmatrix mat) {
    assert(mat.data);
    assert(mat.cols);
    assert(mat.rows);
}

fmatrix fmatrix_create_on_host(int rows, int cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    mat.data = (float*)malloc(fmatrix_size(mat));
    assert(mat.data);
    return mat;
}

fmatrix fmatrix_create_on_device(int rows, int cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    gpuErrchk(
        cudaMalloc((void **)&(mat.data), fmatrix_size(mat))
    );
    return mat;
}

void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_device.data, mat_host.data,
                   fmatrix_size(mat_host),
                   cudaMemcpyHostToDevice
                   )
        );
}

void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_host.data, mat_device.data,
                   fmatrix_size(mat_device),
                   cudaMemcpyDeviceToHost
                   )
        );
}

fmatrix fmatrix_copy_to_host(fmatrix mat_device) {
    fmatrix_assert(mat_device);
    fmatrix mat_host = fmatrix_create_on_host(mat_device.rows, mat_device.cols);
    fmatrix_data_to_host(mat_host,mat_device);
    return mat_host;
}

fmatrix fmatrix_copy_to_device(fmatrix mat_host) {
    fmatrix_assert(mat_host);
    fmatrix mat_device = fmatrix_create_on_device(mat_host.rows, mat_host.cols);
    fmatrix_data_to_device(mat_host,mat_device);
    return mat_device;
}

/** We could do it like this, but it would not set our pointer M.data to 0.
... fmatrix_free_on_host(M)
void fmatrix_free_on_host(fmatrix mat) {
    fmatrix_assert(mat);
  free(mat.data);
  mat.data = 0;
  mat.cols = 0;
  mat.rows = 0;
}
*/

void fmatrix_free_on_host(fmatrix* mat) {
    fmatrix_assert(*mat);
  free(mat->data);
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

void fmatrix_free_on_device(fmatrix* mat) {
    fmatrix_assert(*mat);
  gpuErrchk(cudaFree(mat->data));
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

fmatrix fmatrix_subcolumns(fmatrix M, int a, int b) {
    fmatrix_assert(M);
    fmatrix A = {
        .data = &getfm(M,0,a),
        .cols = b-a,
        .rows = M.rows
    };
    fmatrix_assert(A);
    return A;
}


__host__
__device__
void fmatrix_print(fmatrix mat, int nb){
    if (nb<0 || nb > mat.rows) {
        nb = mat.rows;
    }
    printf("[\n");
    for (int i = 0 ; i < nb; i++){
      for (int j = 0 ; j<mat.cols; j++){
        printf("%f", getfm(mat,i,j));
        if (j+1<mat.cols) {
          printf(",\t");
        }
      }
      if (i+1<nb) {
        printf(";\n");
      }
    }
    if (nb < mat.rows) {
      printf("\n...\n");
    }
  printf("\n]\n");
}

void fmatrix_device_print(fmatrix mat, int nb){
   // allocate copy
   fmatrix tmp = fmatrix_copy_to_host(mat);
   fmatrix_print(tmp,nb);
   fmatrix_free_on_host(&tmp);
}

void fmatrix_device_to_csv(const char* filename, fmatrix mat) {
  // Open file
  FILE* fp = fopen(filename, "w");
  // allocate copy
  fmatrix tmp = fmatrix_copy_to_host(mat);
  for (int i = 0 ; i < tmp.rows; i++){
    for (int j = 0 ; j<tmp.cols; j++){
      // Note: %.15g gives 15 significant digits (full double precision)
      fprintf(fp,"%.15g", getfm(tmp,i,j));
      if (j+1<tmp.cols) {
        fprintf(fp,",");
      }
    }
    fprintf(fp,"\n");
  }
  fmatrix_free_on_host(&tmp);
  // Close file
  fclose(fp);
}

__global__
void fmatrix_create_random_on_device_kernel(fmatrix M) {
    // choose a seed (here: the same each launch)
    unsigned long seed = 0;
    int sequence = 0;
    // first, initialize the random numbers
    curandState state;
    curand_init(seed, sequence, 0, &state);
    for (int i = 0; i < fmatrix_elements(M); ++i) {
        // curand_uniform creates numbers between 0 and 1
        M.data[i] = (curand_uniform(&state)-0.5)*2.0;
    }
}

fmatrix fmatrix_create_random_on_device(int rows, int cols) {
    // Create an uninitialized matrix on the device
    fmatrix M = fmatrix_create_on_device(rows,cols);
    // Call a kernel with a single thread to fill the values
    fmatrix_create_random_on_device_kernel<<<1,1>>>(M);

    return M;
}

/* Count the number of rows and columns in a csv files (without headers) */
void count_elements_in_csv(const char* filename, int* rows, int* cols) {
  // Note: for the sake of convenience, we use some C++ functions here
  using namespace std;

  *rows = 0;
  *cols = 0;
  string row_as_string;
  string value;
  ifstream infile;
  infile.open(filename, ifstream::in);
	if (infile.is_open())
  {
    while (getline(infile, row_as_string, '\n')) {
				istringstream line_stream(row_as_string);
        int tempcols = 0;
        while (getline(line_stream, value, ',')) {
          ++tempcols;
        }
        if (tempcols > *cols) {
           *cols = tempcols;
        }
        ++(*rows);
			}
		infile.close();
	}
	else cout << "Cannot open file." << endl;
}

/** Read the data from a csv file into an fmatrix on the host.
 *  Careful: We assume that the matrix has the right dimensions!
 *  Use count_elements_in_csv(...) to get the dimensions if
 *  unknown.
 */
void fmatrix_fill_from_csv(fmatrix h_M,const char* filename) {
  // Note: for the sake of convenience, we use some C++ functions here
  using namespace std;
  string row_as_string;
  string value;
  ifstream infile;
  infile.open(filename, ifstream::in);
  int row = 0;
	if (infile.is_open())
  {
    while (getline(infile, row_as_string, '\n')) {
				istringstream line_stream(row_as_string);
        int col = 0;
        while (getline(line_stream, value, ',')) {
					getfm(h_M,row,col) = strtod(value.c_str(), NULL);
          ++col;
				}
        ++row;
			}
		infile.close();
	}
	else cout << "Cannot open file." << endl;
}

fmatrix fmatrix_device_from_csv(const char* filename) {
  // first read the file to count the number of elements
  int rows = 0;
  int cols = 0;
  count_elements_in_csv(filename,&rows,&cols);

  // allocate the matrix on the host
  fmatrix h_M = fmatrix_create_on_host(rows,cols);

  // read the data into the host matrix
  fmatrix_fill_from_csv(h_M,filename);

  // copy the matrix to the device
  fmatrix M = fmatrix_copy_to_device(h_M);

  // destroy the host matrix
  fmatrix_free_on_host(&h_M);

  return M;
}

Writing fmatrix.cu


# Fmatrix operations


## Simple Softmax


In [6]:
%%writefile fmatrix_simple_softmax.cu
#include "fmatrix.cuh"
#include <assert.h>
#include <math.h>

#define THREADS_PER_BLOCK 1024

__global__
void fmatrix_simple_softmax_kernel(fmatrix Z) {
    // Each thread works one column of Z
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < Z.cols) {
        float sum = 0.0;
        for (int k = 0; k < Z.rows; k++) {
            getfm(Z, k, idx) = exp(getfm(Z, k, idx));
            sum += getfm(Z, k, idx);
        }

        for (int k = 0; k < Z.rows; k++) {
            getfm(Z, k, idx) /= sum;
        }
    }
}

void fmatrix_simple_softmax(fmatrix Z) {
    fmatrix_assert(Z);

    int threadsPerBlock = Z.cols;
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_simple_softmax_kernel<<< blocksPerGrid, threadsPerBlock >>>(Z);
    gpuErrchk(cudaPeekAtLastError());
    device_synchronize();
}

Writing fmatrix_simple_softmax.cu


## Stable Softmax


In [7]:
%%writefile fmatrix_stable_softmax.cu
#include "fmatrix.cuh"
#include <assert.h>
#include <math.h>

#define THREADS_PER_BLOCK 1024

__global__
void fmatrix_stable_softmax_kernel(fmatrix Z) {
    // Each thread processes one column of Z
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (col < Z.cols) {
        float max_val = getfm(Z, 0, col);
        for (int row = 1; row < Z.rows; row++) {
            float val = getfm(Z, row, col);
            if (val > max_val) {
                max_val = val;
            }
        }

        float sum = 0.0f;
        for (int row = 0; row < Z.rows; row++) {
            float exp_val = expf(getfm(Z, row, col) - max_val);
            getfm(Z, row, col) = exp_val;
            sum += exp_val;
        }

        for (int row = 0; row < Z.rows; row++) {
            getfm(Z, row, col) /= sum;
        }
    }
}

void fmatrix_stable_softmax(fmatrix Z) {
    fmatrix_assert(Z);

    int threadsPerBlock = Z.cols;
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_stable_softmax_kernel<<< blocksPerGrid, threadsPerBlock >>>(Z);
    gpuErrchk(cudaPeekAtLastError());
    device_synchronize();
}

Writing fmatrix_stable_softmax.cu


## Normalization


In [54]:
%%writefile fmatrix_normalization.cu
#include "fmatrix.cuh"
#include <assert.h>
#include <math.h>

#define THREADS_PER_BLOCK 1024

__global__
void compute_mu_sigma_kernel(fmatrix X, fmatrix mu, fmatrix sigma) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < X.rows) {
        // Step 1: Compute mean
        float sum = 0.0f;
        for (int col = 0; col < X.cols; col++) {
            sum += getfm(X, row, col);
        }
        float mean = sum / X.cols;
        getfm(mu, row, 0) = mean; // Store mean

        // Step 2: Compute standard deviation
        float sum_squared_diff = 0.0f;
        for (int col = 0; col < X.cols; col++) {
            float diff = getfm(X, row, col) - mean;
            sum_squared_diff += diff * diff;
        }
        float std_dev = sqrtf(sum_squared_diff / X.cols);
        getfm(sigma, row, 0) = std_dev; // Store standard deviation
    }
}

void compute_mu_sigma(fmatrix X, fmatrix mu, fmatrix sigma) {
    fmatrix_assert(X);
    fmatrix_assert(mu);
    fmatrix_assert(sigma);

    int threadsPerBlock = X.rows;
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK) {
        blocksPerGrid = (threadsPerBlock - 1) / THREADS_PER_BLOCK + 1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    compute_mu_sigma_kernel<<< blocksPerGrid, threadsPerBlock >>>(X, mu, sigma);
    gpuErrchk(cudaPeekAtLastError());
    device_synchronize();
}

__global__
void fmatrix_normalize_kernel(fmatrix X, fmatrix mu, fmatrix sigma) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < X.rows) {
        float mean = getfm(mu, row, 0);
        float std_dev = getfm(sigma, row, 0);

        for (int col = 0; col < X.cols; col++) {
            const float epsilon = 1e-8f; // Small constant to prevent division by zero
            getfm(X, row, col) = (getfm(X, row, col) - mean) / (std_dev + epsilon);
        }
    }
}

void fmatrix_normalize(fmatrix X, fmatrix mu, fmatrix sigma) {
    fmatrix_assert(X);
    fmatrix_assert(mu);
    fmatrix_assert(sigma);

    int threadsPerBlock = X.rows;
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK) {
        blocksPerGrid = (threadsPerBlock - 1) / THREADS_PER_BLOCK + 1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_normalize_kernel<<< blocksPerGrid, threadsPerBlock >>>(X, mu, sigma);
    gpuErrchk(cudaPeekAtLastError());
    device_synchronize();
}

Overwriting fmatrix_normalization.cu


# Testing


## Simple Softmax


### Test extreme values


In [55]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_simple_softmax.cu"

int main() {
    fmatrix Z = fmatrix_device_from_csv("test_M.csv");

    fmatrix_simple_softmax(Z);
    fmatrix_device_to_csv("test_Z.csv", Z);
    fmatrix_free_on_device(&Z);
}

Overwriting test.cu


In [56]:
!nvcc -arch=sm_75 -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [57]:
%%time
import numpy as np

# (column 1: large, column 2: small, column 3: normal)
M = np.array([
    [1e5, -1e3, 1],
    [1e6, -1e4, 2],
    [1e7, -1e5, 3],
], dtype=np.float32)
np.savetxt("test_M.csv", M, delimiter=',')

!./a.out
Z_test = np.loadtxt("test_Z.csv", delimiter=',', ndmin=2)
print(Z_test)

[[       nan        nan 0.09003057]
 [       nan        nan 0.24472848]
 [       nan        nan 0.66524094]]
CPU times: user 5.12 ms, sys: 1.74 ms, total: 6.86 ms
Wall time: 205 ms


As we can see, large and small values in the input matrix result in a `nan` value in the output matrix (after the softmax).

This is normal since our softmax implementation suffers from numerical instability.


### Test against scipy implementation


In [58]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_simple_softmax.cu"

int main() {
    fmatrix Z = fmatrix_device_from_csv("test_M.csv");

    fmatrix_simple_softmax(Z);
    fmatrix_device_to_csv("test_Z.csv", Z);
    fmatrix_free_on_device(&Z);
}

Overwriting test.cu


In [59]:
!nvcc -arch=sm_75 -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [60]:
%%time
import numpy as np
from scipy.special import softmax
from random import randint

nb_tests = 50
max_dimension = 5

for i in range(nb_tests):
  n = randint(1,max_dimension)
  m = randint(1,max_dimension)
  M = np.random.rand(n,m)
  np.savetxt("test_M.csv", M, delimiter=',')

  !./a.out
  Z_test = np.loadtxt("test_Z.csv", delimiter=',', ndmin=2)
  Z = softmax(M, axis=0)
  result = np.allclose(Z, Z_test)

  if not result:
    print("Wrong test result:")
    print("Original:\n", M)
    print("Expected:\n", Z)
    print("Got wrong result:\n", Z_test)
    break

if result:
  print("All random ", nb_tests, " test passed.")

All random  50  test passed.
CPU times: user 233 ms, sys: 48.4 ms, total: 282 ms
Wall time: 10.3 s


Our implementation is correct: all the test (in a setting with stable numerical values) are passed.


## Stable Softmax


### Test extreme values


In [61]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_stable_softmax.cu"

int main() {
    fmatrix Z = fmatrix_device_from_csv("test_M.csv");

    fmatrix_stable_softmax(Z);
    fmatrix_device_to_csv("test_Z.csv", Z);
    fmatrix_free_on_device(&Z);
}

Overwriting test.cu


In [62]:
!nvcc -arch=sm_75 -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [63]:
%%time
import numpy as np

# (column 1: large, column 2: small, column 3: normal)
M = np.array([
    [1e5, -1e3, 1],
    [1e6, -1e4, 2],
    [1e7, -1e5, 3],
], dtype=np.float32)
np.savetxt("test_M.csv", M, delimiter=',')

!./a.out
Z_test = np.loadtxt("test_Z.csv", delimiter=',', ndmin=2)
print(Z_test)

[[0.         1.         0.09003057]
 [0.         0.         0.24472848]
 [1.         0.         0.66524088]]
CPU times: user 5.48 ms, sys: 1.23 ms, total: 6.71 ms
Wall time: 206 ms


As we can see, the problem of having large and small values is fixed.

With the stable implementation we now correctly compute the softmax.


### Test against scipy implementation


In [64]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_stable_softmax.cu"

int main() {
    fmatrix Z = fmatrix_device_from_csv("test_M.csv");

    fmatrix_stable_softmax(Z);
    fmatrix_device_to_csv("test_Z.csv", Z);
    fmatrix_free_on_device(&Z);
}

Overwriting test.cu


In [65]:
!nvcc -arch=sm_75 -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [66]:
%%time
import numpy as np
from scipy.special import softmax
from random import randint

nb_tests = 50
max_dimension = 5

for i in range(nb_tests):
  n = randint(1,max_dimension)
  m = randint(1,max_dimension)
  M = np.random.rand(n,m)
  np.savetxt("test_M.csv", M, delimiter=',')

  !./a.out
  Z_test = np.loadtxt("test_Z.csv", delimiter=',', ndmin=2)
  Z = softmax(M, axis=0)
  result = np.allclose(Z, Z_test)

  if not result:
    print("Wrong test result:")
    print("Original:\n", M)
    print("Expected:\n", Z)
    print("Got wrong result:\n", Z_test)
    break

if result:
  print("All random ", nb_tests, " test passed.")

All random  50  test passed.
CPU times: user 225 ms, sys: 53.3 ms, total: 278 ms
Wall time: 10.6 s


Again, even in this case, all test pass. This means that our implementation of the stable softmax is correct.


## Normalization


### Test against scipy implementation


In [67]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_normalization.cu"

int main() {
    fmatrix X = fmatrix_device_from_csv("test_M.csv");
    fmatrix mu = fmatrix_create_on_device(X.rows, 1);
    fmatrix sigma = fmatrix_create_on_device(X.rows, 1);

    compute_mu_sigma(X, mu, sigma);
    fmatrix_normalize(X, mu, sigma);

    fmatrix_device_to_csv("test_X.csv", X);
    fmatrix_device_to_csv("test_mu.csv", mu);
    fmatrix_device_to_csv("test_sigma.csv", sigma);

    fmatrix_free_on_device(&X);
    fmatrix_free_on_device(&mu);
    fmatrix_free_on_device(&sigma);

    return 0;
}

Overwriting test.cu


In [68]:
!nvcc -arch=sm_75 -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [69]:
%%time
import numpy as np
from scipy.special import softmax
from random import randint

nb_tests = 50
max_dimension = 5

for i in range(nb_tests):
  n = randint(1,max_dimension)
  m = randint(1,max_dimension)
  M = np.random.rand(n, m).astype(np.float32)
  np.savetxt("test_M.csv", M, delimiter=',')

  !./a.out
  X_test = np.loadtxt("test_X.csv", delimiter=',', ndmin=2)
  mu_test = np.loadtxt("test_mu.csv", delimiter=',', ndmin=2)
  sigma_test = np.loadtxt("test_sigma.csv", delimiter=',', ndmin=2)

  mu = np.mean(M, axis=1, keepdims=True)
  sigma = np.std(M, axis=1, keepdims=True)
  epsilon = 1e-8  # Small value to avoid division by zero
  X = (M - mu) / (sigma + epsilon)

  result = np.allclose(X, X_test, atol=1e-6) and \
          np.allclose(mu, mu_test, atol=1e-6) and \
          np.allclose(sigma, sigma_test, atol=1e-6)

  if not result:
    print("Wrong test result:")
    print("Original:\n", M)
    print("Expected:\n", X)
    print("Got wrong result:\n", X_test)
    print("Expected mu:\n", mu)
    print("Got wrong result mu:\n", mu_test)
    print("Expected sigma:\n", sigma)
    print("Got wrong result sigma:\n", sigma_test)
    break

if result:
  print("All random ", nb_tests, " test passed.")

All random  50  test passed.
CPU times: user 267 ms, sys: 37.8 ms, total: 305 ms
Wall time: 10.3 s


All test passed: our implementation of the z-normalization is correct.
